In [2]:
import os
import mysql.connector as connector
from mysql.connector import Error

In [3]:
try:
    connection = connector.connect(
        host='localhost', 
        user=os.environ.get('MYSQL_DB_USER'), 
        password=os.environ.get('MYSQL_DB_PASSWORD'), 
        database='littlelemondb')
    if connection.is_connected():
        print('Connected to mysql server', connection.get_server_info())
        print('Connected to database', connection.database)
        cursor = connection.cursor()
except Error as e:
    print('Error when connecting:', e.msg)

Connected to mysql server 8.0.33
Connected to database littlelemondb


In [7]:
show_tables_query = 'SHOW TABLES';

In [14]:
cursor.execute(show_tables_query)

In [15]:
columns = cursor.column_names
results = cursor.fetchall();

In [16]:
print(columns[0] + ':')
for result in results:
    print(result[0])

Tables_in_littlelemondb:
bookings
customers
employees
menuitems
orderitems
orders
orderstatuses
ordersview


In [17]:
customer_promotion_query = """
SELECT CONCAT(Customers.FirstName, ' ', Customers.LastName) AS CustomerFullName, 
	Customers.Address, Customers.ContactNo, Customers.Email,
    SUM(OrderItems.Quantity * MenuItems.Price) AS BillAmount
FROM Customers
INNER JOIN Bookings ON Customers.CustomerID = Bookings.CustomerID
INNER JOIN Orders ON Bookings.BookingID = Orders.BookingID
INNER JOIN OrderItems ON Orders.OrderID = OrderItems.OrderID
INNER JOIN MenuItems ON OrderItems.ItemID = MenuItems.ItemID
GROUP BY Customers.FirstName, Customers.LastName, Customers.Address, Customers.ContactNo, Customers.Email
HAVING BillAmount > 60
""";

In [23]:
cursor.execute(customer_promotion_query)

In [24]:
columns = cursor.column_names
results = cursor.fetchall();

In [25]:
print(columns)
for result in results:
    print(result)

('CustomerFullName', 'Address', 'ContactNo', 'Email', 'BillAmount')
('Ariana Moon', '631 West 32nd Avenue, Anchorage AK 99503', '(442)000-1332', 'ariana@gmail.com', Decimal('81.30'))


In [26]:
cursor.close()
connection.close()